# Setup

In [1]:
# download red wine quality data
#! mkdir wine
#! wget -P wine https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

# download and unzip Musk v2
# ! mkdir musk
# ! wget -P musk https://archive.ics.uci.edu/ml/machine-learning-databases/musk/clean2.data.Z
# ! uncompress musk/clean2.data.Z

# ! git clone --branch wip/clustering https://github.com/jsokolowska/anomaly-detection.git
# ! mv anomaly-detection ad

In [2]:
# install pyod - not present by default in google colab
! pip install pyod


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
WINE_PATH = "./wine/winequality-red.csv"
MUSK_PATH = "./musk/clean2.data"

In [2]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

def score( y_true, y_pred, y_proba):
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    return precision, recall, f1, roc

# Parameter Tuning

In [9]:
from sklearn.model_selection import GridSearchCV
from anomaly_detection import ClusterBasedAnomalyDetection
from sklearn.cluster import Birch
from sklearn.model_selection import StratifiedKFold
from data_loading import load_wine, load_musk
import pandas as pd

In [10]:
df = pd.DataFrame(index=["musk", "wine"], columns=["birch-lcdof", "birch-cblof", "dbscan-lcdof", "dbscan-cblof"])
# Params for wine
param_grid = {
    "alpha": [0.8, 0.9, 0.95],
    "beta": [3, 5, 10],
    "contamination": [0.1, 0.07, 0.05, 0.03],
    "n_clusters" : [10, 3, 4, 9, 16, 25],
    "threshold": [0.2, 0.5, 0.7]
}
N_SPLITS = 5
X, y = load_wine(WINE_PATH)
skf = StratifiedKFold(n_splits=N_SPLITS, random_state = 91, shuffle = True)

In [ ]:
clustering_algs = [("birch", Birch())]
measures = ["ldcof", "cblof"]
datasets = {"musk": load_musk(MUSK_PATH), "wine": load_wine(WINE_PATH)}

for dataset_name, values in datasets.items():
    X,y = values
    for name, algorithm in clustering_algs:
        for measure in measures:
            cbad = ClusterBasedAnomalyDetection(clustering_estimator=algorithm, dissimilarity_measure=measure)

            search = GridSearchCV(param_grid=param_grid, estimator=cbad, scoring="roc_auc", cv = 5, n_jobs=1)
            search.fit(X, y)
            print(f"[{name}/{measure}/{dataset_name}] Best params: {search.best_params_}")
            print(f"[{name}/{measure}/{dataset_name}] Best score: {search.best_score_}")
            df[f"{name}-{measure}"][dataset_name] = search.best_params_
